# Run analysis script

This gets variables into namespace.

In [ ]:
%run strain_report_dev.py

# Imports

In [ ]:
import seaborn as sns
from scipy.signal import find_peaks

# Keys available

Print keys available for dataframes in signal collection, in alphabetical order

In [ ]:
for key in sorted(strain.signals.keys()):
    print(key)

# mCherry

In [ ]:
sns.heatmap(strain.signals['mCherry/butter'].dropna().head(25))

In [ ]:
foo = strain.signals['mCherry/butter'].dropna()

In [ ]:
fig, ax = plt.subplots(figsize=(10,3))
plt.plot(foo.iloc[7])

In [ ]:
foo.iloc[7]

# Flavin

## Time series

In [ ]:
sns.heatmap(strain.signals['flavin/butter'], cmap='RdBu', robust=True)

In [ ]:
strain.signals['flavin/butter']

In [ ]:
strain.signals['flavin/butter'].iloc[72]

In [ ]:
sns.heatmap(strain.signals['births'])

## Autocorrelation

Display from dataframes

In [ ]:
# ACFs
sns.heatmap(
    strain.signals['flavin/butter_acf'].dropna(),
    cmap='RdBu',
    robust=True
)

In [ ]:
sns.heatmap(
    wildtype.signals['flavin/butter_acf'].dropna(),
    cmap='RdBu',
    robust=True
)

Autocorrelation function (ACF) of single time series, and indicating locations of peaks.

In [ ]:
def acorr(y):
    """Autcorrelation function of single time series"""
    norm = y - np.mean(y)
    var = np.var(y)
    acorr = np.correlate(norm, norm, 'full')[len(norm)-1:]
    acorr = acorr / var / len(norm)
    return acorr

# Choose cell
celliloc = 5

timeaxis = wildtype.signals['flavin/butter'].columns
flavin_ts = wildtype.signals['flavin/butter'].iloc[celliloc].to_numpy()
birth_mask = wildtype.signals['births'].iloc[celliloc].to_numpy()
peaks = wildtype.signals['flavin/butter_peaks'].iloc[celliloc].to_numpy()
peaks_mask = np.ma.make_mask(peaks)

# Plot time series and birth
plt.subplots(figsize=(20,5))
single_birth_plot(timeaxis, flavin_ts, birth_mask=birth_mask)
plt.plot(timeaxis[peaks_mask], flavin_ts[peaks_mask], 'x', color='r')

# Plot autocorrelation function, with peaks indicated
plt.subplots()
acf = acorr(flavin_ts)
acf_peaks, _ = find_peaks(acf)
plt.plot(acf)
plt.plot(acf_peaks, acf[acf_peaks], 'x')

print('Location of ACF peaks (hours):')
print(acf_peaks/12)
print('\n')
print('Location of first ACF peak (hours):')
print(f'{acf_peaks[0] / 12:.3f}')

Proxies for synchrony of time series across population

In [ ]:
# Get median of scaled time series in dataset
med = standardscaler.as_function(
    strain.signals['flavin/butter']).median()

plt.subplots()
plt.plot(med)
plt.title('Median time series')

plt.subplots()
plt.plot(acorr(med))
plt.title('ACF of median time series')
plt.xlabel('lag (time point)')
plt.ylabel('correlation')

fft_freqs, fft_power = fft.as_function(pd.DataFrame(med).T)
plt.subplots()
plt.plot(fft_freqs.to_numpy()[0], fft_power.to_numpy()[0])
plt.title('Fourier spectrum of median time series')
plt.xlabel('frequency')
plt.ylabel('power')

Peaks in population

In [ ]:
# Find peaks in norm_corr, plot on heatmap
from postprocessor.core.processes.findpeaks import findpeaksParameters, findpeaks
acfs = strain.signals['flavin/butter_acf'].dropna()
acfs_peaks = findpeaks.as_function(acfs, prominence = 0.20, width=4)

plt.subplots(figsize=(15,15))
heatmap(
    trace_df=acfs,
    trace_name='ACF',
    buddings_df=acfs_peaks,
    xtick_step=12,
    xlabel='Lag (time point)',
    cmap='RdBu_r',
    cbarlabel='Correlation',
)

In [ ]:
# Plot where acf peaks are -- helps with calibrating findpeaks parameters
pos = 16
plt.plot(acfs.iloc[pos])
p_mask = np.ma.make_mask(acfs_peaks.to_numpy())
plt.plot(acfs.columns[p_mask[pos]], acfs.iloc[pos][p_mask[pos]], 'x')

Distribution of cycle lengths, estimated from ACF.

This is in contrast to using `find_peaks` directly on the time series (flavin, mCherry) and to finding intervals directly from birth times (for CDC lengths)

In [ ]:
# get interval lengths then get just the first one
# (adapted from first line in mask_to_intervals)
def get_first_interval(x):
    list_intervals = np.diff(np.where(np.array(x) > 0))[0]
    # Checks that it is not empty
    if list_intervals.any():
        return list_intervals[0]
    else:
        return np.nan

temp_df = acfs_peaks.apply(
    lambda x: get_first_interval(x), axis=1
)

# draw histogram
fig, ax = plt.subplots()
histogram(
    5 * temp_df.dropna().to_numpy(),
    label='cycle',
    binsize=10,
    plot_title='Distribution of cycle durations (based on ACF)'
)
tick_spacing = 60
ax.xaxis.set_major_locator(ticker.MultipleLocator(tick_spacing))

# XY coordinates of traps

In [ ]:
cell_ids = strain.signals['flavin/butter'].index.to_list()
x_coords = []
y_coords = []
for position, trap, cell_label in cell_ids:
    traps = grouper.tilelocs[position]
    x_coords.append(traps[trap,0])
    y_coords.append(traps[trap,1])

In [ ]:
xy_df = pd.DataFrame([x_coords, y_coords])
xy_df = xy_df.T
xy_df.columns = ['x', 'y']
xy_df.index = strain.signals['flavin/butter'].index

In [ ]:
xy_df

In [ ]:
xy_df.to_csv('XY_coords.csv', index=True)